In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import re
import numpy as np
from loaddata import get_data

def load_model(modelname):
    import os
    os.environ["THEANO_FLAGS"] = "device=cpu"
    from keras.models import model_from_json
    from trainrnn import TDD, SimpleRNNTDD
    with open('%s.json'%modelname) as f:
        src = f.read()
        model = model_from_json(src, custom_objects=dict(TDD=TDD, SimpleRNNTDD=SimpleRNNTDD))
    model.load_weights('%s_weights.h5' % modelname)
    return model


def copymodel(model):
    import keras.models
    newmodel = keras.models.model_from_yaml(model.to_yaml()) 
    # copy the model (model_from_yaml() should compile() according to source code)
    wts=model.get_weights()
    newmodel.set_weights(wts)
    return newmodel



In [ ]:
trajs, trajs_trn, observable = get_data()
mean_pred_var = (trajs - trajs.mean(axis=0)).var()
print trajs.shape

In [ ]:
SEP = '**********************'

from collections import defaultdict, OrderedDict
perfdict_byarch = defaultdict(OrderedDict)
perfdict_bydims = defaultdict(OrderedDict)
BASEDIR='out_loss'
allfiles =  sorted(os.listdir(BASEDIR))
#allfiles = ['small_stacked_longtrain.txt', ]
#allfiles = ['reg_final.txt',]
for fname in allfiles:
    print "Doing", fname
    with open(BASEDIR + '/' + fname) as f:
        s = f.read().split(SEP)
        for sndx in range(1,len(s)):
            clines = s[sndx].strip().split("\n")
            
            specs = clines[0].replace('DOING ','').split(", ")
            if len(specs) == 2:
                arch, dims, reg = specs[0], specs[1], 0
            else:
                arch, dims, reg = specs
                arch += '_' + str(reg)
            val_loss = map(float, re.findall(r'val_loss: ([.0-9]+)', s[sndx]))
            if not val_loss:
                continue
                
            loss = np.array(map(float, re.findall(r' loss: ([.0-9]+)', s[sndx])))
            best_loss_ndx = np.argsort(loss)[0]
            modelnames = re.findall(r'saving model to (.*)_weights.h5', s[sndx])
            if not len(modelnames):
                modelname = clines[-1].replace('Saving ', '')
            else:
                modelname = modelnames[0]
                
            if dims in perfdict_byarch[arch]:
                cdims = dims
                ndx = 2
                while cdims in perfdict_byarch[arch]:
                    cdims = dims + '_v%d'%ndx
                    ndx += 1
                dims = cdims
            minloss = np.array(val_loss)[best_loss_ndx]
            print "- %20s / %10s (minloss:%0.5f, minall:%0.5f)" % (arch, dims, minloss, min(val_loss)), modelname
            cdict = dict(perf=np.array(val_loss), 
                         trainperf=np.array(loss), 
                         modelname=modelname,
                         bestlossperf=minloss)
            perfdict_byarch[arch][dims] = cdict
            perfdict_bydims[dims][arch] = cdict
            
perfdict_byarch = dict(perfdict_byarch.items())
perfdict_bydims = dict(perfdict_bydims.items())

In [ ]:
# Correlation between validation and training loss

if False:
    for k, v in perfdict_byarch.items():
        for k2, v2 in v.items():
            plt.figure()
            p=v2['perf']
            tp=v2['trainperf']
            ndxs = p<np.median(p)
            plt.scatter( tp[ndxs], p[ndxs] , marker='.')
            plt.xlabel('val')
            plt.ylabel('train')
            plt.title('%s-%s'%(k, k2))        
        

In [ ]:
with plt.style.context('ggplot'):
    for perfdict in (perfdict_byarch, perfdict_bydims):
        fig = plt.figure(figsize=(8,3*len(perfdict)))
        for ndx, k1 in enumerate(sorted(perfdict.keys())):
            plt.subplot(len(perfdict),1,  ndx+1 )
            for k2, v in perfdict[k1].items():
                vals = v['perf']
                plt.plot(vals, label=k2) # plt.plot(perfdict.values()[0])
            mlen = max(len(v['perf']) for v in perfdict[k1].values())
            #mlen= 200
            plt.xlim([0, mlen])
            plt.plot(np.zeros(mlen) + mean_pred_var, 'k--', label='Mean')
            plt.gca().legend(loc='right', bbox_to_anchor=(1.5, .5) )
            plt.title(k1)
del perfdict


In [ ]:
cmodel = perfdict_byarch['arch=rnn2_reg=0']['dims=700']
cmodelname = cmodel['modelname']
model = load_model(cmodelname)

In [ ]:
trajs_tst       = trajs[int(0.9*len(trajs)):]
observable_tst  = observable[int(0.9*len(trajs)):]
all_predictions = model.predict(trajs_tst)
#all_predictions = trajs.mean(axis=0)[None,:,:]

print (observable_tst - all_predictions).var()
errs = ((observable_tst - all_predictions)**2).mean(axis=2).mean(axis=0)
with plt.style.context('ggplot'):
    plt.plot(errs)
    plt.ylim([0, 1.3*plt.ylim()[1]])

print "Mean error: %0.5f" % errs.mean()
print "Reported  : %0.5f" % cmodel['bestlossperf']

In [ ]:
SHOW_NUM_PERTS = 5
plt.figure(figsize=(14,5*SHOW_NUM_PERTS))

for sampndx, sampnumber in enumerate(np.random.choice(len(observable_tst), SHOW_NUM_PERTS)):
    plt.subplot(SHOW_NUM_PERTS,2,sampndx*2+1)

    vmax = observable_tst[sampnumber].max() * 1.2
    plt.imshow(observable_tst[sampnumber], vmin=0, vmax=vmax, cmap='jet_r', interpolation='none',aspect='auto')
    plt.colorbar()
    plt.title('Actual (pert %d)' % sampnumber)

    plt.subplot(SHOW_NUM_PERTS,2,sampndx*2+2)
    pred_in = trajs_tst[sampnumber].copy()[None,:,:].copy()
    pred_in[:,1:,:] = 0.
    pred_out = model.predict(pred_in)
    plt.imshow(pred_out[0], vmin=0, vmax=vmax, cmap='jet_r', interpolation='none',aspect='auto')
    plt.colorbar()
    plt.title('Predicted (pert %d)' % sampnumber)
    
    #print sampndx, sampnumber, (observable_tst[sampnumber] - pred_out).var()


In [ ]:
SHOW_NUM_VARS = 10
sz = int(np.sqrt(SHOW_NUM_VARS)+1)
plt.figure(figsize=(4*sz, 4*sz))
timendx = -1
#timendx = 3
mostvariedndxs = np.argsort(observable_tst[:,-1,:].var(axis=0))
for ndx in range(SHOW_NUM_VARS):
    plt.subplot(sz,sz,ndx+1)
    mostvaried=mostvariedndxs[-1-ndx]
    
    plt.scatter(observable_tst[:,timendx,mostvaried], all_predictions[:,timendx,mostvaried], 1, 'k')
    corr = np.corrcoef(observable_tst[:,timendx,mostvaried], all_predictions[:,timendx,mostvaried])[0,1]
    plt.gca().annotate('c=%0.3f'%corr, xy=(0.1, 0.9), xycoords='axes fraction')
    plt.xlabel('Reality')
    plt.axis('square')
    minlim = min(plt.xlim()[0],plt.ylim()[0])
    maxlim = min(plt.xlim()[1],plt.ylim()[1])
    plt.xlim([minlim, maxlim])
    plt.ylim([minlim, maxlim])
    plt.hold('on')
    plt.plot([minlim,maxlim],[minlim,maxlim], '--')
    plt.ylabel('Prediction')
plt.tight_layout()

# Performance vs. accuracy

In [ ]:
def plot_perf_dict(perf_dict, do_logscale=True):
    import matplotlib.cm as cm
    import matplotlib
    import seaborn as sns
    with plt.style.context('seaborn-whitegrid', after_reset=True):
        sns.set(font_scale=4)
        colors = cm.rainbow(np.linspace(0, 1, len(perf_dict)))
        for ndx, (k,v) in enumerate(perf_dict.iteritems()):
            plt.scatter(v[0], v[1], s=150, c=colors[ndx], label=k, edgecolors='none')

        #plt.plot(plt.xlim(), [mean_pred_var,mean_pred_var], 'k--', label='Mean')
        plt.gca().legend(loc='right', bbox_to_anchor=(1.65, .5) )
        if do_logscale:
            plt.gca().set_xscale("log", nonposx='clip')

        #plt.title("Performance vs. Accuracy")
        plt.xlabel('Operations/step')
        plt.ylabel('Error')

carch = 'rnn2_reg=0' # rnn4_stacked'
#carch = 'rnn4'
complex_perf_dict = OrderedDict()
fulldims = trajs.shape[2]
complex_perf_dict['Actual'] = (fulldims**3, 0)
for k,v  in perfdict_byarch['arch=%s'%carch].iteritems():
    dims = int(k.split("=")[1].split("_")[0])
    perf = v['bestlossperf']
    complex_perf_dict[k] = (dims**2, perf)
    
plt.figure(figsize=(10,10))
plot_perf_dict(complex_perf_dict)

# L1 performance-accuracy tradeoff

In [ ]:
reg_models = perfdict_byarch.keys()

scaling = {}
best_scaling = {}
ALL_CUTOFFS = [3e-2, 2e-2, 1e-2, 7.5e-3, 5e-3, 2.5e-3, 1e-3, ]
err_cutoff = 0.95
for r in reg_models:
    
    if r[-5:] == 'reg=0': #  or r[-9:] == 'reg=4e-06':
        continue
        
    cmodel = perfdict_byarch[r]['dims=500']
    lasterr = cmodel['bestlossperf']
    print "Reported  : %0.5f (%s)" % (lasterr, r)
    
    cmodelname = cmodel['modelname']
    cmodelobj = load_model(cmodelname)
    weights = cmodelobj.layers[1].get_weights()
    origmx = weights[1]
    
    
    for cutoff in sorted(ALL_CUTOFFS):
        mx = origmx.copy()
        mx[np.abs(mx) < cutoff] = 0.0
        weights[1] = mx
        cmodelobj.layers[1].set_weights(weights)

        nnz = np.sum(~np.isclose(mx, 0))
        #scaling[(r,cutoff)] = (nnz, lasterr)
        #continue

        all_predictions = cmodelobj.predict(trajs[int(0.9*len(trajs)):])
        err = ((observable[int(0.9*len(trajs)):] - all_predictions)**2).mean()

        print "Mean error: %0.5f" % err, ' ratio: %0.3f' % (lasterr/err), 'nnz:', nnz, ' tot:', mx.shape[0]*mx.shape[1], ' cutoff=', cutoff
            
        scaling[(r,cutoff)] = (nnz, err)
        if lasterr/err > err_cutoff:
            print "  Saving to best scaling"
            best_scaling[r] = (nnz, err)
    print 

In [ ]:
if False:
    """
    perfdict = {}
    for k,v in scaling.iteritems():
        #print k[1]
        if k[1]==0.005:
            perfdict[k[0].split("=")[2]] = v
    """
    #d1=complex_perf_dict.copy()
    #del d1['full']
    d1=OrderedDict(Actual=complex_perf_dict['Actual'])
    d1.update(best_scaling)
    print best_scaling
    plt.figure(figsize=(10,10))
    plot_perf_dict(d1)
    plt.ylim([-0.02,0.06])

In [ ]:
"""
perfdict = {}
for k,v in scaling.iteritems():
    #print k[1]
    if k[0][-5:] == 'reg=0':
        continue
    if k[1]==0.01:
        perfdict[k[0].split("=")[2]] = v
        
"""
perfdict=best_scaling

do_logscale=True
plt.figure(figsize=(10,10))
import matplotlib.cm as cm
import matplotlib
import seaborn as sns
with plt.style.context('seaborn-whitegrid', after_reset=True):
    sns.set(font_scale=4)
    #colors = cm.rainbow(np.linspace(0, 1, len(perf_dict)))
    cdict = complex_perf_dict.copy()
    actual = cdict['Actual']
    del cdict['Actual']
    xs, ys = zip(*cdict.values())
    plt.plot(xs, ys, linestyle='None', marker='o', mew=0, markersize=13, color='red', label='Dimensionality')
    for d, (x, y) in cdict.items():
        txt = d.replace('dims=','d=')
        plt.text(x*1.3, y, txt, fontsize=20, va='center')
    
    xs, ys = zip(*perfdict.values())
    plt.plot(xs, ys, linestyle='None', marker='+', mew=3, markersize=20, color='blue', label='Comp cost penalty')

    plt.plot([actual[0],], [actual[1],], linestyle='None', marker='*', mew=2, markersize=20, color='k', label='Full model')
    
    #plt.plot(plt.xlim(), [mean_pred_var,mean_pred_var], 'k--', label='Mean')
    plt.gca().legend(loc='right', bbox_to_anchor=(1.9, .5) )
    if do_logscale:
        plt.gca().set_xscale("log", nonposx='clip')
    plt.xlim([1e2, plt.xlim()[1]*10])
    plt.ylim([-.005, 0.05])
    #plt.title("Performance vs. Accuracy")
    plt.xlabel('Operations/step')
    plt.ylabel('Error')



# Goal directed

In [ ]:
print perfdict_byarch.keys()
#perfdict_byarch['arch=rnn4']['dims=50']
m1   = load_model(perfdict_byarch['arch=rnn2_reg=0']['dims=50']['modelname'])
m1v2 = load_model(perfdict_byarch['arch=rnn2_reg=0']['dims=50_v2']['modelname'])
#m2 = load_model(perfdict_byarch['arch=rnn2_reg=0']['dims=500']['modelname'])

m2   = load_model(perfdict_byarch['arch=rnn2_reg=0']['dims=500']['modelname'])
m2v2 = load_model(perfdict_byarch['arch=rnn2_reg=0']['dims=500_v2']['modelname'])


In [ ]:
p1   = m1.predict(trajs_tst)
p1v2 = m1v2.predict(trajs_tst)
p2   = m2.predict(trajs_tst)
p2v2 = m2v2.predict(trajs_tst)
#p2 = m2.predict(trajs_tst)

In [ ]:
err1 = ((p1 - observable_tst)**2).mean(axis=1).mean(axis=0)[280]
err1v2 = (p1v2 - observable_tst[:,:,280][:,:,None]).var()
err2 = ((p2 - observable_tst)**2).mean(axis=1).mean(axis=0)[280]
err2v2 = (p2v2 - observable_tst[:,:,280][:,:,None]).var()
plt.figure(figsize=(10,10))
offset=0.5

import pandas as pd
pd=pd.DataFrame([[50, err1, err1v2],[500,err2,err2v2]], columns=['dim','Global','Goal'])
pd = pd.set_index('dim')
pd
#, index='Dimensions')
#print pd
with plt.style.context('seaborn-white'):
    sns.set(font_scale=3)
    pd.plot(kind='bar')
    plt.ylabel('Error')
    plt.xlabel('Compressed Dimensions')
    labels = plt.gca().get_xticklabels()
    plt.setp(labels, rotation=0)
#    #plt.bar([1,2], [err1[280], err1v2])
#    plt.bar([1,2], [err1[280]  , err2[280]], label='Global')
#    plt.bar([1+offset,2+offset], [err1v2, err2v2], label='Goal')
#    plt.legend()

In [ ]:
import matplotlib.cm as cm
import matplotlib
import seaborn as sns
plt.figure(figsize=(10,8))
with plt.style.context('seaborn-whitegrid', after_reset=True):
    sns.set(font_scale=4)

    edgeerrs = ((p1 - observable_tst)**2).mean(axis=1).mean(axis=0)
    print np.sort(edgeerrs)[-10:]
    print edgeerrs[280]
    worstedge = np.argsort(edgeerrs)[-1]
    print worstedge
    ndx = 300
    plt.plot(p1[ndx,:,worstedge], label='Global')
    plt.plot(p2[ndx,:,worstedge], label='Goal-focused')
    plt.plot(observable_tst[ndx,:,worstedge], label='Actual')
    plt.gca().legend(loc='right', bbox_to_anchor=(1.65, .5) )
    plt.xlabel('Time')
    plt.ylabel('State')


In [ ]:
from pypower import case300
d=case300.case300()
bus_idmap = { busid : ix for ix, busid in enumerate(d['bus'][:,0].astype('int')) }
edgelist = [(bus_idmap[s], bus_idmap[e]) for s, e in d['branch'][:,0:2].astype('int')]

import igraph
G=igraph.Graph(edgelist)
layout_force = G.layout_kamada_kawai()
layout_tree = G.layout_reingold_tilford()
color_dict = { 0: 'green', 1: 'blue', 2: 'red' , 3: 'pink', 4: 'cyan'}
colors = np.zeros(len(G.vs), dtype='int')
for l in d['bus']:
    colors[bus_idmap[l[0]]] = int(l[1])
colornames = [color_dict[c] for c in colors]

In [ ]:
cm=plt.get_cmap('jet')
bboxsize=(0,0,300,300)

#edgecolor=[cm(c) for c in edgeerrs/edgeerrs.max()]
edgecolor = 1
igraph.plot(G, layout=layout_force, vertex_size=2,  vertex_color=colornames, vertex_frame_color=colornames, edge_color=edgecolor,
           bbox=bboxsize)

In [ ]:
n1 = G.neighbors(106)
n1 = set(list(n1) + sum([G.neighbors(i) for i in n1], []))
n1 = set(list(n1) + sum([G.neighbors(i) for i in n1], []))
#n1 = set(list(n1) + sum([G.neighbors(i) for i in n1], []))
colornames2 = ['red' if i in n1 else 'blue' for i in range(len(colors))]
igraph.plot(G, layout=layout_force, vertex_size=2,  vertex_color=colornames2, vertex_frame_color=colornames2, edge_color=edgecolor,
           bbox=bboxsize)
print sorted(list(n1))

In [ ]:
cmodel = perfdict_byarch['arch=rnn2_invstacked']['dims=3']
cmodelobj = load_model(cmodel['modelname'])

In [ ]:
print len(cmodelobj.layers)
inMx  = cmodelobj.layers[0].get_weights()[0].T
outMx = cmodelobj.layers[4].get_weights()[0]

plotMx=inMx
edgecolor=[color_dict[c] for c in np.argmax(np.abs(plotMx), axis=0)]
igraph.plot(G, layout=layout_force, vertex_size=2,  vertex_color=colornames, vertex_frame_color=colornames, edge_color=edgecolor,
           bbox=bboxsize)


In [ ]:
from IPython.display import display
cm=plt.get_cmap('PRGn')
#plotMx = outMx
plotMx = inMx

scale = np.max(np.abs(plotMx))
print 'scale', scale
for vals in plotMx.copy():
    vals /= scale
    print len(vals)
    vals /= 2
    vals /= np.max(np.abs(vals))
    #vals = np.zeros(len(edgelist))+.1
    vals += 0.5
    edgecolor=[cm(c) for c in vals]
    #print len(edgecolor)
    display(igraph.plot(G, layout=layout_force, vertex_size=2,  
                        vertex_color=colornames, vertex_frame_color=colornames, 
                        edge_width=2, edge_color=edgecolor, bbox=bboxsize))


In [ ]:
cm=plt.get_cmap('jet')
vals  = np.max(np.abs(plotMx), axis=0)
vals /= vals.max()

print len(vals)
edgecolor=[cm(c) for c in vals]
#print len(edgecolor)
display(igraph.plot(G, layout=layout_tree, vertex_size=2,  
                    vertex_color=colornames, vertex_frame_color=colornames, 
                    edge_width=2, edge_color=edgecolor, bbox=bboxsize))


In [ ]:
igraph.plot(G, layout=layout_tree, vertex_size=2,  vertex_color=colornames, vertex_frame_color=colornames,
           bbox=np.array(bboxsize)*.7)
